# Review output from MAAP DPS for ABoVE Boreal Biomass Project: Summer 2022 update
* Boreal biomass cloud-optimized geotiff (COG) tiles are brought together in a MosaicJSON file that is mapped with Folium
* to update the AGB tindex master csv and its corresponding MosaicJson, run _build_mosaic_json.ipynb_

Paul Montesano, Laura Duncanson (PI), Nathan Thomas

## Attention: adjust path
##### Path to your icesat2_boreal/lib dir - clone the icesat2_boreal GitHub repository
https://github.com/lauraduncanson/icesat2_boreal.git

In [1]:
ICESAT2_BOREAL_REPO_PATH = '/projects/icesat2_boreal' # /projects/Developer/icesat2_boreal/lib
ICESAT2_BOREAL_LIB_PATH = ICESAT2_BOREAL_REPO_PATH + '/lib'

In [2]:
# For some reason this is needed to get s3fs to work in ExtractUtils
# this upgrades to 0.3.4 even though we already specify this version in requirements_main...
!pip install s3fs --upgrade

You should consider upgrading via the '/opt/conda/bin/python3.7 -m pip install --upgrade pip' command.


In [3]:
try:
    import geopandas as gpd
except ModuleNotFoundError:
    print('Need to pip install packages...')
    !pip install -U -r $ICESAT2_BOREAL_REPO_PATH/dps/requirements_main.txt
    
import geopandas
import pandas as pd
import os
import json
import collections
import numpy as np
import sys
import s3fs
import matplotlib.pyplot as plt
sys.path.append(ICESAT2_BOREAL_LIB_PATH)
sys.path.append('/projects/Developer/icesat2_boreal/lib')
import maplib_folium
import ExtractUtils
from folium import TileLayer
print("Importing packages complete.")

/projects/.local/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


Importing packages complete.


## Set up paths to MosaicJson files needed to map output

In [4]:
AGB_prelim_tindex_master_fn = 's3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/AGB_tindex_master.csv'
AGB_prelim_mosaic_json_fn = 's3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/AGB_tindex_master_mosaic.json'

AGB_v2_tindex_master_fn = 's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/v2/AGB_tindex_master.csv'  # <-- 07 indicates the summer update
AGB_v2_mosaic_json_fn   = 's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/v2/AGB_tindex_master_mosaic.json'

AGB_spring2022_tindex_master_fn = 's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/06/AGB_tindex_master.csv'  # <-- 07 indicates the summer update
AGB_spring2022_mosaic_json_fn   = 's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/06/AGB_tindex_master_mosaic.json'

AGB_summer2022_tindex_master_fn = 's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/07/AGB_tindex_master.csv'  # <-- 07 indicates the summer update
AGB_summer2022_mosaic_json_fn   = 's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/07/AGB_tindex_master_mosaic.json'

AGB_fall2022_tindex_master_fn = 's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/AGB/fall2022/map_boreal_2022_v3/11/AGB_tindex_master.csv'
AGB_fall2022_mosaic_json_fn = 's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/AGB/fall2022/map_boreal_2022_v3/11/AGB_tindex_master_mosaic.json'

AGB_fall2022_noground_tindex_master_fn = 's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/AGB/fall2022/with_atl03_rh/map_boreal_2022_rh_noground_v1/12/AGB_tindex_master.csv'
AGB_fall2022_noground_mosaic_json_fn = 's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/AGB/fall2022/with_atl03_rh/map_boreal_2022_rh_noground_v1/12/AGB_tindex_master_mosaic.json'

#HLS_tindex_master_fn = 's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/HLS_test_redo/spring2022/HLS_tindex_master.csv'
#HLS_mosaic_json_fn   = 's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/HLS_test_redo/spring2022/HLS_tindex_master_mosaic.json'

#HLS_tindex_c2020_master_fn = f's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/HLS/fall2022/HLS_stack_2022_v2/HLS_tindex_master.csv'
#HLS_mosaic_c2020_json_fn   = f's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/HLS/fall2022/HLS_stack_2022_v2/HLS_tindex_master_mosaic.json'

# Final HLS comp for c2020
HLS_tindex_c2020_master_fn = f's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/HLS/c2020/HLS_stack_2022_v2/HLS_tindex_mastercsv'
HLS_mosaic_c2020_json_fn = f's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/HLS/c2020/HLS_stack_2022_v2/HLS_tindex_master_mosaic.json'

Topo_tindex_master_fn  = 's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/Topo_tindex_master.csv'
Topo_mosaic_json_fn  = 's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/Topo_tindex_master_mosaic.json'

#ATL08_filt_tindex_master_fn = 's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/ATL08_filt_tindex_master.csv'
ATL08_filt_tindex_master_fn = 's3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/fall2022/ATL08_filt_tindex_master.csv'
#ATL08_filt_tindex_json_fn= "/projects/my-public-bucket/DPS_tile_lists/ATL08_filt_tindex_master.json"

#LC_mosaic_json_fn = 's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/09/LC_tindex_master_mosaic.json'

# Final LC mosaic for c2020
LC_mosaic_json_fn = 's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/LC/LC_tindex_master_mosaic.json'

boreal_tile_index_path = '/projects/shared-buckets/nathanmthomas/boreal_tiles_v003.gpkg' 

In [5]:
import matplotlib
matplotlib.rcParams['figure.figsize'] = [30, 10]
import importlib
import ExtractUtils
importlib.reload(ExtractUtils)

Ht_fall2022_noground_tindex_master_fn = 's3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/fall2022/AGB_tindex_master_height.csv'
AGB_fall2022_noground_tindex_master_fn = 's3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/fall2022/AGB_tindex_master_noground.csv'
AGB_fall2022_ground_tindex_master_fn = 's3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/fall2022/AGB_tindex_master_ground.csv'

BAD_TILE_LIST = [3540,3634,3728,3823,3916,4004,41995,41807,41619]
if False:
    Ht_mosaic_json_fn_noground, CURRENT_tindex_matches_gdf = ExtractUtils.build_mosaic_json(Ht_fall2022_noground_tindex_master_fn, 
                                                                                          boreal_tile_index_path = '/projects/shared-buckets/nathanmthomas/boreal_tiles_v003.gpkg', 
                                                                                           BAD_TILE_LIST = BAD_TILE_LIST, 
                                                                                           cols_list = ['tile_num','s3_path','local_path'])

    AGB_mosaic_json_fn_noground, CURRENT_tindex_matches_gdf = ExtractUtils.build_mosaic_json(AGB_fall2022_noground_tindex_master_fn, 
                                                                                          boreal_tile_index_path = '/projects/shared-buckets/nathanmthomas/boreal_tiles_v003.gpkg', 
                                                                                           BAD_TILE_LIST = BAD_TILE_LIST, 
                                                                                           cols_list = ['tile_num','s3_path','local_path'])

    AGB_mosaic_json_fn_ground, CURRENT_tindex_matches_gdf = ExtractUtils.build_mosaic_json(AGB_fall2022_ground_tindex_master_fn, 
                                                                                          boreal_tile_index_path = '/projects/shared-buckets/nathanmthomas/boreal_tiles_v003.gpkg', 
                                                                                           BAD_TILE_LIST = BAD_TILE_LIST, 
                                                                                           cols_list = ['tile_num','s3_path','local_path'])

In [6]:
# Get all boreal tiles
boreal_tile_index = geopandas.read_file(boreal_tile_index_path)

#fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(15,10))
#boreal_tile_index["tile_num"] = boreal_tile_index["tile_num"].astype(int)
#boreal_tile_index.plot(column='tile_num', linewidth=0.1, legend=True, ax=ax, cmap='Spectral')

In [7]:
# Get the tindex of the AGB COGs for mapping
tindex_master = pd.read_csv(AGB_fall2022_tindex_master_fn, storage_options={'anon':True})
tindex_master[tindex_master.tile_num == 11].local_path.to_list()

['/projects/my-private-bucket/dps_output/run_boreal_biomass_quick_v2_ubuntu/map_boreal_2022_v3/2022/11/16/13/59/29/451625/boreal_agb_202211161668607096_0011.tif']

In [8]:
if False:
    # Write tindex list to private-bucket
    pd.read_csv(HLS_tindex_master_fn, storage_options={'anon':True}).to_csv('s3://maap-ops-workspace/nathanmthomas/DPS_tile_lists/HLS_tindex_master.csv')
    pd.read_csv(Topo_tindex_master_fn, storage_options={'anon':True}).to_csv('s3://maap-ops-workspace/nathanmthomas/DPS_tile_lists/Topo_tindex_master.csv')
    pd.read_csv(ATL08_filt_tindex_master_fn, storage_options={'anon':True}).to_csv('s3://maap-ops-workspace/nathanmthomas/DPS_tile_lists/ATL08_filt_tindex_master.csv')

### Summary of intermediate (ATL08 filt, HLS, Topo) and AGB output

In [ ]:
ATL08filt_TILES_NEEDED = ExtractUtils.GET_TILES_NEEDED(DPS_DATA_TYPE = 'ATL08_filt', \
                                           tindex_master_fn = ATL08_filt_tindex_master_fn,\
                                                boreal_tile_index_path = boreal_tile_index_path)

In [ ]:
TOPO_TILES_NEEDED = ExtractUtils.GET_TILES_NEEDED(DPS_DATA_TYPE = 'Topo', \
                                           tindex_master_fn = Topo_tindex_master_fn,\
                                                boreal_tile_index_path = boreal_tile_index_path)

In [ ]:
HLS_TILES_NEEDED = ExtractUtils.GET_TILES_NEEDED(DPS_DATA_TYPE = 'HLS', \
                                           tindex_master_fn = HLS_tindex_master_fn,\
                                                 bad_tiles = [], \
                                                boreal_tile_index_path = boreal_tile_index_path)

In [ ]:
AGB_TILES_NEEDED = ExtractUtils.GET_TILES_NEEDED(DPS_DATA_TYPE = 'AGB', \
                                                tindex_master_fn = AGB_fall2022_tindex_master_fn,\
                                                 bad_tiles = [], \
                                                boreal_tile_index_path = boreal_tile_index_path)

In [9]:
BAD_TILE_LIST = [3540,3634,3728,3823,3916,4004,41995,41807,41619]

# For the tindex_master, convert it into vector tiles that show the tiles we have
cols_list = ['tile_num','s3_path','local_path']

# Select the rows we have results for
tile_index_matches = boreal_tile_index.merge(tindex_master[~tindex_master['tile_num'].isin(BAD_TILE_LIST)][cols_list], how='right', on='tile_num')
tile_index_matches = tile_index_matches[tile_index_matches['s3_path'].notna()]
tile_index_matches.shape

(4714, 7)

In [10]:
# Testing overview resampling builds for COGs
if False:
    in_fn = '/projects/my-public-bucket/boreal_agb_202205041651623844_0042.tif'
    out_fn = '/projects/my-public-bucket/boreal_agb_202205041651623844_0042-TEST_GDAL_OVERVIEWS.tif'
    !gdal_translate -co OVERVIEW_RESAMPLING=AVERAGE -of COG $in_fn $out_fn 

# View s3 map layers

In [11]:
# Setup the mosaic tiling
tiler_base = "https://jqsd6bqdsf.execute-api.us-west-2.amazonaws.com/" #titiler.maap-project.org
tiler_mosaic =  "".join([tiler_base, "mosaicjson/tiles/{z}/{x}/{y}"])

In [12]:
# Fill in your favorite tile numbers that will highlight in red 
CHECK_TILE_LIST = [] #[3585,4108,1037,1549]
if False:
    df = gpd.read_file('/projects/my-public-bucket/DPS_tile_lists/HLS_irregular_tiles.gpkg')
    CHECK_TILE_LIST = df.tile_num.to_list()

In [13]:
import importlib
import maplib_folium
importlib.reload(maplib_folium)

<module 'maplib_folium' from '/projects/Developer/icesat2_boreal/lib/maplib_folium.py'>

### Create tile layers of various MS comp layers available for mapping

In [14]:
# Choose a MS band to map
MS_BANDNAME = 'NBR2'
MS_BANDMIN=0.25
MS_BANDMAX=0.45 
MS_BANDNUM=13 
MS_BANDCOLORBAR='nipy_spectral'

#
# Specify the mosaic jsons of various composites
#
# initial HLS compfor c2020 built in fall 2022
HLS_mosaic_fall2022_json_fn = f's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/HLS/fall2022/HLS_stack_2022_v2/HLS_tindex_master_mosaic.json'
HLS_mosaic_fall2022_FIX_json_fn = f's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/HLS/fall2022_fix_irregulars/HLS_stack_2022_v2/12/HLS_tindex_master_mosaic.json'

# Final HLS comp for c2020
HLS_mosaic_c2020_json_fn = f's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/HLS/c2020/HLS_stack_2022_v2/HLS_tindex_master_mosaic.json'

# Final HLS comp for c2015
HLS_mosaic_c2015_json_fn = f's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/HLS/c2015/HLS_stack_2022_v2/11/HLS_tindex_master_mosaic.json' 

# Test of HLS comp for late winter c2020
HLS_mosaic_c2020latewinter_json_fn = 's3://maap-ops-workspace/shared/nathanmthomas/DPS_tile_lists/HLS/c2020latewinter/HLS_stack_2022_v2/HLS_tindex_master_mosaic.json'

# Tile Layers
mscomp_fall2022_FIX_tiles_layer = TileLayer(
            tiles= f"{tiler_mosaic}?url={HLS_mosaic_fall2022_FIX_json_fn}&rescale={MS_BANDMIN},{MS_BANDMAX}&bidx={MS_BANDNUM}&colormap_name={MS_BANDCOLORBAR}",
            opacity=1,
            name="fixed tiles HLS comp",
            attr="MAAP",
            overlay=True
        )

mscomp_c2020_tiles_layer = TileLayer(
            tiles= f"{tiler_mosaic}?url={HLS_mosaic_c2020_json_fn}&rescale={MS_BANDMIN},{MS_BANDMAX}&bidx={MS_BANDNUM}&colormap_name={MS_BANDCOLORBAR}",
            opacity=1,
            name="final c2020 HLS comp",
            attr="MAAP",
            overlay=True
        )

mscomp_c2015_tiles_layer = TileLayer(
            tiles= f"{tiler_mosaic}?url={HLS_mosaic_c2015_json_fn}&rescale={MS_BANDMIN},{MS_BANDMAX}&bidx={MS_BANDNUM}&colormap_name={MS_BANDCOLORBAR}",
            opacity=1,
            name="final c2015 HLS comp",
            attr="MAAP",
            overlay=True
        )

mscomp_c2020latewinter_tiles_layer = TileLayer(
            tiles= f"{tiler_mosaic}?url={HLS_mosaic_c2020latewinter_json_fn}&rescale={MS_BANDMIN},{MS_BANDMAX}&bidx={MS_BANDNUM}&colormap_name={MS_BANDCOLORBAR}",
            opacity=1,
            name="Late Winter c2020 HLS comp",
            attr="MAAP",
            overlay=True
        )

# Dict for specifying a different MS comp band to map
MS_BAND_kwargs = {
    'MS_BANDNAME': MS_BANDNAME,
    'MS_BANDMIN' : MS_BANDMIN ,
    'MS_BANDMAX' : MS_BANDMAX , 
    'MS_BANDNUM' : MS_BANDNUM, 
    'MS_BANDCOLORBAR' : MS_BANDCOLORBAR,
}

In [15]:
MAX_AGB = 125

# AGB_ground_tiles_layer = TileLayer(
#             tiles= f"{tiler_mosaic}?url={AGB_mosaic_json_fn_ground}&rescale=0,{MAX_AGB}&bidx=1&colormap_name=viridis",
#             opacity=1,
#             name="AGB summer 2022",
#             attr="MAAP",
#             overlay=True
#         )


AGB_summer2022_tiles_layer = TileLayer(
            tiles= f"{tiler_mosaic}?url={AGB_summer2022_mosaic_json_fn}&rescale=0,{MAX_AGB}&bidx=1&colormap_name=viridis",
            opacity=1,
            name="AGB summer 2022",
            attr="MAAP",
            overlay=True
        )
AGB_spring2022_tiles_layer = TileLayer(
            tiles= f"{tiler_mosaic}?url={AGB_spring2022_mosaic_json_fn}&rescale=0,{MAX_AGB}&bidx=1&colormap_name=viridis",
            opacity=1,
            name="AGB spring 2022",
            attr="MAAP",
            overlay=True
        )
AGB_v2_tiles_layer = TileLayer(
            tiles= f"{tiler_mosaic}?url={AGB_v2_mosaic_json_fn}&rescale=0,{MAX_AGB}&bidx=1&colormap_name=viridis",
            opacity=1,
            name="AGB v2 2022",
            attr="MAAP",
            overlay=True
        )
MAX_HT = 30
Ht_mosaic_json_fn_noground = 's3://maap-ops-workspace/shared/lduncanson/DPS_tile_lists/fall2022/AGB_tindex_master_height_mosaic.json'
Ht_tiles_layer  = TileLayer(
            tiles= f"{tiler_mosaic}?url={Ht_mosaic_json_fn_noground}&rescale=0,{MAX_HT}&bidx=1&colormap_name=magma",
            opacity=1,
            name="Height 2020",
            attr="MAAP",
            overlay=True
        )

In [ ]:
maplib_folium.MAP_DPS_RESULTS(tiler_mosaic, 
                              boreal_tile_index[~boreal_tile_index['tile_num'].isin(BAD_TILE_LIST)], #bad_tiles 
                              #geopandas.read_file( '/projects/my-public-bucket/analyze_agb/footprints_terrapulse-pub-data_boreal-tcc-unclipped-1995.gpkg'), # rm tiles [444, 576, 695, 805, 913,1127, 1197]
                              tile_index_matches, 
                              boreal_tile_index[boreal_tile_index['tile_num'].isin(CHECK_TILE_LIST)], 
                              MATCH_TILES_NAME='AGB tiles',
                              CHECK_TILES_NAME='Tiles to check',
                              mosaic_json_dict = 
                                  {     ######### PRODUCT LAYERS #######
                                        #---NASA ABoVE Aboveground Boreal Woody Biomass Density
                                        'agb_mosaic_json_s3_fn':       AGB_fall2022_mosaic_json_fn, #AGB_mosaic_json_fn_noground
                                        'topo_mosaic_json_s3_fn':      Topo_mosaic_json_fn,
                                        'mscomp_mosaic_json_s3_fn':    HLS_mosaic_c2020_json_fn,
                                        ######### ANALYSIS LAYERS ######
                                        'worldcover_json_s3_fn':       LC_mosaic_json_fn,  
                                        'tp_standage2020_json_s3_fn':  None,
                                        'tp_tcc2020_json_s3_fn':       None,
                                        'tp_tcc2020slope_json_s3_fn':  None,
                                        'tp_tcc2020pvalue_json_s3_fn': None, 
                                    },
                              max_AGB_display = MAX_AGB, max_AGBSE_display = 20,
                              map_width=1250, map_height=500,
                                                           mscomp_rgb_dict = None,
                            **MS_BAND_kwargs,
                            SHOW_WIDGETS=False,
                            ADD_TILELAYER = Ht_tiles_layer  
                             )

Sites to spot check: 

N 58.41332 E 7.25588  
- notice barren is showing high AGB  
- does masking out barren LC interfere with AGB prediction over actual barren that isnt class'd as such?  
- If this is the case - then we DONT want to remove BARREN prior to prediction - rather - just mask out after the fact - that way, small barren extents wont be horribly wrong due to near non-existant training

N 58.15207 E 6.85672

In [ ]:
FOCAL_TILE = 11
m=maplib_folium.map_tile_atl08(FOCAL_TILE, tiler_mosaic, boreal_tile_index, ATL08_filt_tindex_master_fn = ATL08_filt_tindex_master_fn,
                             mosaic_json_dict = {'agb_mosaic_json_s3_fn': AGB_fall2022_mosaic_json_fn}, DO_NIGHT=False, map_width=1600, map_height=500, OVERVIEW_MAP=False,
                            max_AGB_display = 150, max_AGBSE_display = 20)
m

In [ ]:
MISSING_TILES = [3082, 2918, 40664, 2745,      # AK & NW Ca
                42809,                         # NE Sib Beringia
                1854, 1789, 1321, 36238, 1262, # Hudson Bay and Quebec
                1505, 1392, 34210, 33975,      # Southern edge,  mostly below 45N - might not have ATL08 processed
                37, 10, 148, 169, 446,         # Scand - 10 very little land so no ATL08?
                162, 629,                             # Western Russia
                27365,                                # Crimea
                1761,2651,2970,3417,3604, 4080        # Central and Eastern Siberia
                ]
print(MISSING_TILES)
len(MISSING_TILES)

In [ ]:
MISSING_TILES_for_redo = []
MISSING_ATL08 = []
MISSING_HLS = []
MISSING_Topo = []

ATL08_filt_tindex = pd.read_csv(ATL08_filt_tindex_master_fn, storage_options={'anon':True})
HLS_tindex = pd.read_csv(HLS_tindex_master_fn, storage_options={'anon':True})
Topo_tindex = pd.read_csv(Topo_tindex_master_fn, storage_options={'anon':True})

AGB_tindex = pd.read_csv(AGB_tindex_master_fn, storage_options={'anon':True})

for t in MISSING_TILES:

    if t in ATL08_filt_tindex.tile_num.to_list() and HLS_tindex.tile_num.to_list() and Topo_tindex.tile_num.to_list():
        MISSING_TILES_for_redo.append(t)
        
        # Just confirm that this tile is in fact not in the AGB tindex< if so, flag it with a print out.
        if t in AGB_tindex.tile_num.to_list():
            print(f"Tile {t} is in the AGB tindex, so maybe its not showing?")
    else:
        if t not in ATL08_filt_tindex.tile_num.to_list():
            MISSING_ATL08.append(t)
        if t not in HLS_tindex.tile_num.to_list():
            MISSING_HLS.append(t)
        if t not in Topo_tindex.tile_num.to_list():
            MISSING_Topo.append(t)
            
print(f"AGB does NOT exist for these {len(MISSING_TILES_for_redo)} tiles, but ATL08 filt, HLS, & Topo tiles do:\n{MISSING_TILES_for_redo}\nRe-do AGB run for these tiles.")

In [ ]:
MISSING_HLS

In [ ]:
MISSING_Topo